In [ ]:
import json
import os
import numpy
import pandas
import pyarrow
import sys

from datetime import date
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

In [ ]:
PACKAGE_ID = 7520845

In [ ]:
#
def create_transfer_packages_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            company_deliveries.delivery_type,
            company_deliveries.updated_at,
            metrc_deliveries.received_datetime,
            metrc_transfer_packages.package_id,
            metrc_transfer_packages.package_label,
            metrc_transfer_packages.package_payload.packagetype,
            metrc_transfer_packages.shipped_quantity,
            metrc_transfer_packages.package_payload.shipperwholesaleprice,
            metrc_transfer_packages.received_quantity,
            metrc_transfer_packages.package_payload.receiverwholesaleprice,
            metrc_transfers.transfer_payload.shipmenttransactiontype,
            metrc_transfers.shipment_type_name,
            metrc_transfers.shipper_facility_name,
            metrc_transfers.shipper_facility_license_number,
            metrc_deliveries.recipient_facility_name,
            metrc_deliveries.recipient_facility_license_number,
            metrc_transfers.*,
            metrc_transfer_packages.*
        from
            metrc_transfer_packages
            inner join metrc_deliveries on metrc_transfer_packages.delivery_row_id = metrc_deliveries.id
            inner join metrc_transfers on metrc_deliveries.transfer_row_id = metrc_transfers.id
            inner join company_deliveries on metrc_transfers.id = company_deliveries.transfer_row_id
            inner join companies on company_deliveries.company_id = companies.id
        where
            True
            and metrc_transfer_packages.package_id = '{package_id}'
    """

transfer_packages_by_package_id_query = create_transfer_packages_by_package_id_query(PACKAGE_ID)
transfer_packages_by_package_id_dataframe = pandas.read_sql_query(transfer_packages_by_package_id_query, engine)
transfer_packages_by_package_id_dataframe

In [ ]:
transfer_packages_by_package_id_dataframe.to_dict('records')[0]

In [ ]:
# Find packages in data warehouse by package_label.

packages_by_package_label_query = create_queries.create_packages_by_package_label_query('1A40603000075FF000000096')
packages_by_package_label_dataframe = pandas.read_sql_query(packages_by_package_label_query, engine)
packages_by_package_label_dataframe

In [ ]:
# Find packages in data warehouse by package_id.

def create_packages_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            metrc_packages.type,
            metrc_packages.package_type,
            metrc_packages.product_category_name,
            metrc_packages.product_name,
            metrc_packages.package_id,
            metrc_packages.package_label,
            metrc_packages.quantity,
            metrc_packages.unit_of_measure
        from
            metrc_packages
            left outer join companies on metrc_packages.company_id = companies.id
        where
            True
            and metrc_packages.package_id = '{package_id}'
    """

packages_by_package_id_query = create_packages_by_package_id_query(PACKAGE_ID)
packages_by_package_id_dataframe = pandas.read_sql_query(packages_by_package_id_query, engine)
packages_by_package_id_dataframe

In [ ]:
def create_sales_transactions_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            metrc_sales_receipts.type,
            metrc_sales_receipts.created_at,
            metrc_sales_receipts.updated_at,
            metrc_sales_receipts.last_modified_at,
            metrc_sales_receipts.is_final,
            metrc_sales_transactions.is_deleted,
            metrc_sales_receipts.receipt_id,
            metrc_sales_receipts.receipt_number,
            metrc_sales_receipts.sales_datetime,
            metrc_sales_transactions.package_id,
            metrc_sales_transactions.package_label,
            metrc_sales_transactions.unit_of_measure,
            metrc_sales_transactions.quantity_sold,
            metrc_sales_transactions.total_price
        from
            metrc_sales_transactions
            left outer join metrc_sales_receipts on metrc_sales_transactions.receipt_row_id = metrc_sales_receipts.id
            left outer join companies on metrc_sales_receipts.company_id = companies.id
        where
            True
            and metrc_sales_transactions.package_id = '{package_id}'
    """

sales_transactions_by_package_id_query = create_sales_transactions_by_package_id_query(PACKAGE_ID)
sales_transactions_by_package_id_dataframe = pandas.read_sql_query(sales_transactions_by_package_id_query, engine)
sales_transactions_by_package_id_dataframe.sort_values('sales_datetime')

In [ ]:
def create_sales_transactions_is_deleted_counts_query():
    return f"""
        select
            metrc_sales_transactions.is_deleted,
            count(*)
        from
            metrc_sales_transactions
        where
            True
        group by
            metrc_sales_transactions.is_deleted
    """

sales_transactions_is_deleted_counts_query = create_sales_transactions_is_deleted_counts_query()
sales_transactions_is_deleted_counts_dataframe = pandas.read_sql_query(sales_transactions_is_deleted_counts_query, engine)
sales_transactions_is_deleted_counts_dataframe

In [ ]:
def create_sales_transactions_is_deleted_query():
    return f"""
        select
            companies.identifier,
            metrc_sales_transactions.is_deleted,
            metrc_sales_transactions.package_id,
            metrc_sales_transactions.package_label
        from
            metrc_sales_transactions
            left outer join metrc_sales_receipts on metrc_sales_transactions.receipt_row_id = metrc_sales_receipts.id
            left outer join companies on metrc_sales_receipts.company_id = companies.id
        where
            metrc_sales_transactions.is_deleted = True
    """

sales_transactions_is_deleted_query = create_sales_transactions_is_deleted_query()
sales_transactions_is_deleted_dataframe = pandas.read_sql_query(sales_transactions_is_deleted_query, engine)
sales_transactions_is_deleted_dataframe

In [ ]:
def create_sales_transactions_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            metrc_sales_receipts.type,
            metrc_sales_receipts.last_modified_at,
            metrc_sales_receipts.is_final,
            metrc_sales_transactions.is_deleted,
            metrc_sales_receipts.receipt_id,
            metrc_sales_receipts.receipt_number,
            metrc_sales_receipts.sales_datetime,
            metrc_sales_transactions.package_id,
            metrc_sales_transactions.package_label,
            metrc_sales_transactions.unit_of_measure,
            metrc_sales_transactions.quantity_sold,
            metrc_sales_transactions.total_price
        from
            metrc_sales_transactions
            left outer join metrc_sales_receipts on metrc_sales_transactions.receipt_row_id = metrc_sales_receipts.id
            left outer join companies on metrc_sales_receipts.company_id = companies.id
        where
            True
            and metrc_sales_transactions.package_id = '{package_id}'
    """

sales_transactions_by_package_id_query = create_sales_transactions_by_package_id_query(18149539)
sales_transactions_by_package_id_dataframe = pandas.read_sql_query(sales_transactions_by_package_id_query, engine)
sales_transactions_by_package_id_dataframe.sort_values('sales_datetime')